# Imports

In [1]:
import sys
print(sys.version)

3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]


In [2]:
import pandas as pd
import numpy as np
import statistics

In [3]:
!pip install statsbombpy
!pip install pandas
!pip install tidyverse
!pip install r2d3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 3.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement r2d3 (from versions: none)
ERROR: No matching distribution found for r2d3


In [4]:
from statsbombpy import sb

# Get event dataframes

In [5]:
# get all events in a season
events = sb.competition_events(
    country="Spain",
    division= "La Liga",
    season="2017/2018",
    gender="male",
    split = True
)
type(events)

/usr/local/lib/python3.9/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


collections.defaultdict

In [6]:
comps = sb.competitions()


/usr/local/lib/python3.9/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [7]:
### This block may take a few mins to run ###

# Get all La Liga competitions (17 total)

import warnings
warnings.filterwarnings('ignore')

# List of dicts containing all La Liga competition data
la_liga = []

# List of ids (e.g. "La Liga 2018/2019")
ids = []
j = 1
for i, row in comps.iterrows():
  # Skip any competitions that aren't La Liga
  if row["competition_name"] != "La Liga":
    continue
  
  print("Reading row " + str(j) + "/17")
  j += 1
  # Pull the competition events via the statsbombpy API
  curr_event = sb.competition_events(
      country=row["country_name"],
      division=row["competition_name"],
      season=row["season_name"],
      gender=row["competition_gender"],
      split = True
  )
  id = row["competition_name"] + " " + row["season_name"] 
  ids.append(id)
  la_liga.append({id: curr_event})

Reading row 1/17
Reading row 2/17
Reading row 3/17
Reading row 4/17
Reading row 5/17
Reading row 6/17
Reading row 7/17
Reading row 8/17
Reading row 9/17
Reading row 10/17
Reading row 11/17
Reading row 12/17
Reading row 13/17
Reading row 14/17
Reading row 15/17
Reading row 16/17
Reading row 17/17


In [8]:
# df with all pass events
passes_df = events["passes"]
# events.head()

# Top lineups and starters (by id)

In [9]:
print(ids)
#print(ids[0])

['La Liga 2020/2021', 'La Liga 2019/2020', 'La Liga 2018/2019', 'La Liga 2017/2018', 'La Liga 2016/2017', 'La Liga 2015/2016', 'La Liga 2014/2015', 'La Liga 2013/2014', 'La Liga 2012/2013', 'La Liga 2011/2012', 'La Liga 2010/2011', 'La Liga 2009/2010', 'La Liga 2008/2009', 'La Liga 2007/2008', 'La Liga 2006/2007', 'La Liga 2005/2006', 'La Liga 2004/2005']


In [10]:
#Put this inside a loop for all La Liga seasons
from collections import Counter, defaultdict

#starters = la_liga[0]["La Liga 2020/2021"]["starting_xis"]
winners = []
all_starters = []
id_names = {}
id_pos = {}
all_starting_players = []


for i, comp in enumerate(la_liga):
  # Get all starting lineups from the season data
  starters = comp[ids[i]]["starting_xis"]
  teams = starters["possession_team"]
  
  # Get team that won the season
  data = Counter(teams)
  #print(data)
  winner = max(teams, key=data.get)
  starting_players = defaultdict(int)

  winners.append({ids[i]: winner})
  curr_starters = starters[starters["team"] == winner]["tactics"].to_dict()
  # starting_xi = defaultdict(int)
  # Find the most common starting lineup for the season to use for our analysis
  for lineup in curr_starters.values():
    players = [p["player"] for p in lineup["lineup"]]
    positions = [p["position"]['name'] for p in lineup["lineup"]]
    player_ids = [str(players[k]['id']) for k in range(len(players))]
    
    # add id and names to id_names
    for j in players:
      id_names[str(j['id'])] = j['name']
      
      starting_players[j['id']] += 1
    id_pos.update(zip(player_ids, positions))

    # p_ids = sorted([str(x["id"]) for x in players])
    # starting_xi[' '.join(p_ids)] += 1

  # get top 11 each season
  s = sorted(starting_players.items(), key=lambda x:x[1], reverse=True)[:11]
  all_starting_players.append({ids[i]: s})

# for starter in all_starting_players:
#   print(starter.values())

  # starting_xi = sorted(starting_xi.items(), key=lambda x:x[1], reverse=True)
  # all_starters.append({ids[i]: starting_xi[:1][0][0].split()})

# all_starters contains a list of dicts with the season ID as the key and the 
# value is a list of the most frequent starting 11 player IDs
# all_starters[0]
# positions

# all_starting_players[1]
# s
#all_starting_players is dict, key = season ID, value = list of tuples (player ID, num_starts)



In [11]:
id_pos

{'20055': 'Goalkeeper',
 '32480': 'Left Center Back',
 '5213': 'Right Center Back',
 '6826': 'Left Center Back',
 '5477': 'Right Wing',
 '5211': 'Left Back',
 '5203': 'Center Defensive Midfield',
 '39073': 'Right Center Midfield',
 '30486': 'Left Midfield',
 '5487': 'Left Center Forward',
 '5503': 'Right Wing',
 '43728': 'Right Center Back',
 '6947': 'Center Defensive Midfield',
 '24841': 'Center Attacking Midfield',
 '4447': 'Left Midfield',
 '5492': 'Left Center Back',
 '21881': 'Right Wing Back',
 '8118': 'Center Defensive Midfield',
 '3501': 'Left Wing',
 '6590': 'Goalkeeper',
 '6379': 'Right Center Midfield',
 '30756': 'Right Wing Back',
 '17304': 'Left Back',
 '22390': 'Right Wing',
 '6374': 'Right Back',
 '5470': 'Right Center Midfield',
 '11392': 'Left Center Midfield',
 '8206': 'Right Center Midfield',
 '5246': 'Center Forward',
 '25006': 'Right Wing',
 '5674': 'Right Back',
 '13599': 'Left Center Midfield',
 '11532': 'Right Center Back',
 '6998': 'Right Center Midfield',
 '63

In [12]:
for pos in id_pos:
  if id_pos[pos] == 'Goalkeeper':
    id_pos[pos] = True
  elif id_pos[pos] != 'Goalkeeper':
    id_pos[pos] = False
  # id_pos[pos] = (id_pos[pos] is 'Goalkeeper')
id_pos

{'20055': True,
 '32480': False,
 '5213': False,
 '6826': False,
 '5477': False,
 '5211': False,
 '5203': False,
 '39073': False,
 '30486': False,
 '5487': False,
 '5503': False,
 '43728': False,
 '6947': False,
 '24841': False,
 '4447': False,
 '5492': False,
 '21881': False,
 '8118': False,
 '3501': False,
 '6590': True,
 '6379': False,
 '30756': False,
 '17304': False,
 '22390': False,
 '6374': False,
 '5470': False,
 '11392': False,
 '8206': False,
 '5246': False,
 '25006': False,
 '5674': False,
 '13599': False,
 '11532': False,
 '6998': False,
 '6332': False,
 '6616': False,
 '8668': False,
 '3109': False,
 '8652': True,
 '5542': False,
 '5216': False,
 '6402': False,
 '6849': False,
 '6400': False,
 '5506': False,
 '6609': False,
 '3726': False,
 '10802': False,
 '6196': False,
 '4320': False,
 '11094': False,
 '10609': False,
 '4691': True,
 '3362': False,
 '4324': False,
 '7068': False,
 '22102': False,
 '3508': False,
 '20131': False,
 '27318': False,
 '3958': False,
 '11302'

# Average Locaton df

In [13]:
# function to calculate the average location of a player given player_id and an event dataframe
# from 'Regular Play' only
def avgloc(id,event_df):
  events = event_df.loc[(event_df['player_id'] == id) & (event_df['play_pattern'] == 'Regular Play')].reset_index(drop=True)
  locations = events['location']
  return [sum(x)/len(x) for x in zip(*locations)]

In [14]:
# Run function avgloc on starts

#top_starts = [x[0] for x in starts]
#num_starts = [x[1] for x in starts]
#print(all_starters[0][ids[0]])
dfs = []
complete_passes_dfs ={}
for i in range(len(all_starting_players)):
  top_starts = list(all_starting_players[i].values())[0]
  # print(top_starts)
  num_starts = [i[1] for i in top_starts] 
  # print(num_starts)
  passes_df = la_liga[i][ids[i]]["passes"]

  # get the Barcelona complete passes
  complete_passes_df = passes_df.loc[(passes_df['team'] == 'Barcelona') & passes_df["pass_outcome"].isna()].reset_index(drop=True)
  complete_passes_dfs[ids[i]] = complete_passes_df
  # call avgloc on each player
  avglocs = [avgloc(int(id[0]), complete_passes_df) for id in top_starts]
  season = [ids[i]]*11
  top_starts_id = [str(id[0]) for id in top_starts]
  df = pd.DataFrame(list(zip(season,top_starts_id,num_starts, avglocs)), columns = ['season', 'player_id', 'season_starts', 'average_location'])
  max_start = df['season_starts'].max()
  new_row = pd.DataFrame({'season':ids[i], 'player_id': 1111111111, 'season_starts': max_start, 'average_location': [[120, 40]], 'player_from': 'Shot','row_id':11}, index=[11])
  df['row_id'] = df.reset_index(drop=True).index
  df.append(new_row)
  

  dfs.append(df)
  
# add a new row for shot as 12th player
  dfs.append(new_row)
  
df = pd.concat(dfs)
# print(complete_passes_df.columns)

df

,season,player_id,season_starts,average_location,row_id,player_from
0,La Liga 2020/2021,5211,33,"[64.3328934010153, 13.335837563451763]",0,NaN
1,La Liga 2020/2021,5503,33,"[76.28396674584323, 45.30712589073629]",1,NaN
2,La Liga 2020/2021,8118,32,"[60.19215876089059, 39.63969022265244]",2,NaN
3,La Liga 2020/2021,5203,30,"[60.64644268774705, 41.794762845849824]",3,NaN
4,La Liga 2020/2021,20055,29,"[11.704034582132566, 39.61152737752163]",4,NaN
...,...,...,...,...,...,...
7,La Liga 2004/2005,5558,6,"[54.10434782608695, 40.78623188405797]",7,NaN
8,La Liga 2004/2005,20131,6,"[57.34043715846993, 43.96338797814207]",8,NaN
9,La Liga 2004/2005,19298,6,"[77.50799999999997, 34.16133333333332]",9,NaN
10,La Liga 2004/2005,5216,5,"[64.98205128205126, 40.01794871794873]",10,NaN


In [127]:
id_names[1111111111] = 'Shot'
# id_names
df['player_from'] = df['player_id'].map(id_names)
df['is_goalie'] = df['player_id'].map(id_pos)
df['is_goalie'] = df['is_goalie'].fillna(0).astype(int)
df_starting = df

,season,player_id,season_starts,average_location,row_id,player_from,is_goalie
0,La Liga 2020/2021,5211,33,"[64.3328934010153, 13.335837563451763]",0,Jordi Alba Ramos,0
1,La Liga 2020/2021,5503,33,"[76.28396674584323, 45.30712589073629]",1,Lionel Andrés Messi Cuccittini,0
2,La Liga 2020/2021,8118,32,"[60.19215876089059, 39.63969022265244]",2,Frenkie de Jong,0
3,La Liga 2020/2021,5203,30,"[60.64644268774705, 41.794762845849824]",3,Sergio Busquets i Burgos,0
4,La Liga 2020/2021,20055,29,"[11.704034582132566, 39.61152737752163]",4,Marc-André ter Stegen,1
...,...,...,...,...,...,...,...
7,La Liga 2004/2005,5558,6,"[54.10434782608695, 40.78623188405797]",7,Rafael Márquez Álvarez,0
8,La Liga 2004/2005,20131,6,"[57.34043715846993, 43.96338797814207]",8,Xavier Hernández Creus,0
9,La Liga 2004/2005,19298,6,"[77.50799999999997, 34.16133333333332]",9,Samuel Eto''o Fils,0
10,La Liga 2004/2005,5216,5,"[64.98205128205126, 40.01794871794873]",10,Andrés Iniesta Luján,0


# To Do

*   Count completed passes from each player to recipient
*   Add shot events to pass events -> edit the average location code




In [ ]:
df_starting.to_csv('df_starting.csv', index=False, encoding='utf-8-sig')

NameError: ignored

In [15]:
# Dictionary and dataframe of passer and recipient pairs
all_season_passes = pd.DataFrame()
for key, value in complete_passes_dfs.items():
  passes = defaultdict(int)
  passers = value['player']
  recipients = value['pass_recipient']
  merged_list = list(zip(passers, recipients))

  for element in merged_list:
    passes[element] += 1

  dfPass = pd.DataFrame([[k[0],k[1],v,key] for k,v in passes.items()]) \
            .rename(columns={0:'Passing',1:'Receiving',2:'Frequency',3:'Season'})
  all_season_passes = all_season_passes.append(dfPass)



print(all_season_passes.head(5))
all_season_passes.tail(5)

                           Passing                        Receiving  \
0   Lionel Andrés Messi Cuccittini         Sergio Busquets i Burgos   
1         Sergio Busquets i Burgos         Moriba Kourouma Kourouma   
2         Moriba Kourouma Kourouma  Ronald Federico Araújo da Silva   
3  Ronald Federico Araújo da Silva            Marc-André ter Stegen   
4            Marc-André ter Stegen                  Clément Lenglet   

   Frequency             Season  
0        256  La Liga 2020/2021  
1         47  La Liga 2020/2021  
2         26  La Liga 2020/2021  
3         85  La Liga 2020/2021  
4        168  La Liga 2020/2021  


,Passing,Receiving,Frequency,Season
240,Henrik Larsson,Rafael Márquez Álvarez,1,La Liga 2004/2005
241,Henrik Larsson,Samuel Eto''o Fils,2,La Liga 2004/2005
242,Henrik Larsson,Andrés Iniesta Luján,1,La Liga 2004/2005
243,Andrés Iniesta Luján,Henrik Larsson,1,La Liga 2004/2005
244,Henrik Larsson,Lionel Andrés Messi Cuccittini,1,La Liga 2004/2005


In [16]:
# df with all shot events
shots_df = events["shots"]
shots_df.info()
shots_df['shot_outcome'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971 entries, 0 to 970
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     971 non-null    object 
 1   index                  971 non-null    int64  
 2   period                 971 non-null    int64  
 3   timestamp              971 non-null    object 
 4   minute                 971 non-null    int64  
 5   second                 971 non-null    int64  
 6   type                   971 non-null    object 
 7   possession             971 non-null    int64  
 8   possession_team        971 non-null    object 
 9   play_pattern           971 non-null    object 
 10  team                   971 non-null    object 
 11  player                 971 non-null    object 
 12  position               971 non-null    object 
 13  location               971 non-null    object 
 14  duration               971 non-null    float64
 15  relate

array(['Saved', 'Blocked', 'Goal', 'Off T', 'Saved to Post', 'Wayward',
       'Post', 'Saved Off Target'], dtype=object)

In [ ]:
# Finding average location of all players at shot event

# Copy avgloc function but remove play pattern constraint
def shotavgloc(id,event_df):
  events = event_df.loc[(event_df['player_id'] == id)].reset_index(drop=True)
  locations = events['location']
  return [sum(x)/len(x) for x in zip(*locations)]

# Copy player/location df but with locations reflecting shooting events
avgshotlocs = [shotavgloc(id, shots_df) for id in top_starts]
shooters = pd.DataFrame(list(zip(top_starts,num_starts, avgshotlocs)), columns = ['player_id', 'season_starts', 'average_location'])
shooters['player_from'] = df['player_id'].map(player_dict)
shooters

,player_id,season_starts,average_location,player_from
0,20055,35,[],Marc-André ter Stegen
1,5503,32,"[102.9055555555556, 39.2489898989899]",Lionel Andrés Messi Cuccittini
2,5246,30,"[108.75267857142856, 38.16428571428572]",Luis Alberto Suárez Díaz
3,5470,29,"[100.0, 44.18461538461539]",Ivan Rakitić
4,5213,28,"[110.9388888888889, 41.322222222222216]",Gerard Piqué Bernabéu
5,5211,28,"[107.55333333333334, 29.680000000000003]",Jordi Alba Ramos
6,5203,28,"[110.0, 39.480000000000004]",Sergio Busquets i Burgos
7,5216,24,"[100.3, 31.11428571428571]",Andrés Iniesta Luján
8,6379,23,"[104.875, 41.00000000000001]",Sergi Roberto Carnicer
9,5492,23,"[110.025, 38.699999999999996]",Samuel Yves Umtiti


In [ ]:
# Shots and goals of each player
players = []
for player in shooters['player_from']:
  df = shots_df.loc[shots_df['player'] == player]
  shots = df.shape[0]
  goals = df.loc[df['shot_outcome'] == 'Goal'].shape[0]
  players.append((player,shots,goals))
shots = pd.DataFrame(list(players),
               columns =['Player', 'Shots',"Goals"])
shots

,Player,Shots,Goals
0,Marc-André ter Stegen,0,0
1,Lionel Andrés Messi Cuccittini,198,34
2,Luis Alberto Suárez Díaz,112,23
3,Ivan Rakitić,26,1
4,Gerard Piqué Bernabéu,18,2
5,Jordi Alba Ramos,15,2
6,Sergio Busquets i Burgos,5,1
7,Andrés Iniesta Luján,14,1
8,Sergi Roberto Carnicer,8,1
9,Samuel Yves Umtiti,8,1


In [ ]:
# manipulate shots dataframe to stack with pass dataframe
shots_man = shots.copy()
shots_man['Passing'] = shots_man['Player']
shots_man['Receiving'] = 'Shot'
shots_man['Frequency'] = shots_man['Shots'] * 1.5
shots_man = shots_man[['Passing', 'Receiving', 'Frequency']]
shots_man

,Passing,Receiving,Frequency
0,Marc-André ter Stegen,Shot,0.0
1,Lionel Andrés Messi Cuccittini,Shot,297.0
2,Luis Alberto Suárez Díaz,Shot,168.0
3,Ivan Rakitić,Shot,39.0
4,Gerard Piqué Bernabéu,Shot,27.0
5,Jordi Alba Ramos,Shot,22.5
6,Sergio Busquets i Burgos,Shot,7.5
7,Andrés Iniesta Luján,Shot,21.0
8,Sergi Roberto Carnicer,Shot,12.0
9,Samuel Yves Umtiti,Shot,12.0


In [ ]:
# stacking pass and shot dataframes together

df_pass_shot = pd.concat([dfPass, shots_man])
df_pass_shot

,Passing,Receiving,Frequency
0,Gerard Piqué Bernabéu,Samuel Yves Umtiti,191.0
1,Samuel Yves Umtiti,Marc-André ter Stegen,157.0
2,Marc-André ter Stegen,Samuel Yves Umtiti,146.0
3,Samuel Yves Umtiti,Jordi Alba Ramos,263.0
4,Samuel Yves Umtiti,Andrés Iniesta Luján,145.0
...,...,...,...
19,Gerard Deulofeu Lázaro,Shot,12.0
20,Aleix Vidal Parreu,Shot,12.0
21,Denis Suárez Fernández,Shot,16.5
22,Yerry Fernando Mina González,Shot,4.5


In [ ]:
# pass interactions between starting 11
mask1 = df_pass_shot['Passing'].isin(df_starting['player_from'])
df_starting_pass = df_pass_shot.loc[mask1]

mask2 = df_starting_pass['Receiving'].isin(df_starting['player_from'])
df_starting_pass = df_starting_pass.loc[mask2]

df_starting_pass



,Passing,Receiving,Frequency
0,Gerard Piqué Bernabéu,Samuel Yves Umtiti,191.0
1,Samuel Yves Umtiti,Marc-André ter Stegen,157.0
2,Marc-André ter Stegen,Samuel Yves Umtiti,146.0
3,Samuel Yves Umtiti,Jordi Alba Ramos,263.0
4,Samuel Yves Umtiti,Andrés Iniesta Luján,145.0
...,...,...,...
6,Sergio Busquets i Burgos,Shot,7.5
7,Andrés Iniesta Luján,Shot,21.0
8,Sergi Roberto Carnicer,Shot,12.0
9,Samuel Yves Umtiti,Shot,12.0


In [ ]:
# calculating passing probabilities
df_starting_pass['Passing_Sum_Frequency'] = df_starting_pass.groupby('Passing')['Frequency'].transform('sum')
df_starting_pass['Pass Probability'] = df_starting_pass['Frequency'] / df_starting_pass['Passing_Sum_Frequency']
df_starting_pass

,Passing,Receiving,Frequency,Passing_Sum_Frequency,Pass Probability
0,Gerard Piqué Bernabéu,Samuel Yves Umtiti,191.0,1242.0,0.153784
1,Samuel Yves Umtiti,Marc-André ter Stegen,157.0,1187.0,0.132266
2,Marc-André ter Stegen,Samuel Yves Umtiti,146.0,766.0,0.190601
3,Samuel Yves Umtiti,Jordi Alba Ramos,263.0,1187.0,0.221567
4,Samuel Yves Umtiti,Andrés Iniesta Luján,145.0,1187.0,0.122157
...,...,...,...,...,...
6,Sergio Busquets i Burgos,Shot,7.5,1458.5,0.005142
7,Andrés Iniesta Luján,Shot,21.0,1032.0,0.020349
8,Sergi Roberto Carnicer,Shot,12.0,1128.0,0.010638
9,Samuel Yves Umtiti,Shot,12.0,1187.0,0.010110


In [ ]:
# calling player id and location for each passing and receiving player:

df_starting_pass = pd.merge(df_starting_pass, df_starting, left_on='Passing', right_on='player_from', how='left')
df_starting_pass = pd.merge(df_starting_pass, df_starting, left_on='Receiving', right_on='player_from', how='left')
df_starting_pass = df_starting_pass.drop(['player_from_x', 'player_from_y'], axis=1)
df_starting_pass = df_starting_pass.loc[:, ~df_starting_pass.columns.duplicated()]
df_starting_pass

,Passing,Receiving,Frequency,Passing_Sum_Frequency,Pass Probability,player_id_x,season_starts_x,average_location_x,row_id_x,player_id_y,season_starts_y,average_location_y,row_id_y
0,Gerard Piqué Bernabéu,Samuel Yves Umtiti,191.0,1242.0,0.153784,5213,28,"[40.62254601226996, 53.52331288343562]",4,5492,23,"[41.16050724637682, 25.78876811594206]",9
1,Samuel Yves Umtiti,Marc-André ter Stegen,157.0,1187.0,0.132266,5492,23,"[41.16050724637682, 25.78876811594206]",9,20055,35,"[11.75150684931507, 40.30493150684933]",0
2,Marc-André ter Stegen,Samuel Yves Umtiti,146.0,766.0,0.190601,20055,35,"[11.75150684931507, 40.30493150684933]",0,5492,23,"[41.16050724637682, 25.78876811594206]",9
3,Samuel Yves Umtiti,Jordi Alba Ramos,263.0,1187.0,0.221567,5492,23,"[41.16050724637682, 25.78876811594206]",9,5211,28,"[58.5618181818182, 13.467972027972031]",5
4,Samuel Yves Umtiti,Andrés Iniesta Luján,145.0,1187.0,0.122157,5492,23,"[41.16050724637682, 25.78876811594206]",9,5216,24,"[65.958273381295, 24.63075539568345]",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Sergio Busquets i Burgos,Shot,7.5,1458.5,0.005142,5203,28,"[56.009207161125325, 37.920204603580565]",6,1111111111,35,"[120, 40]",11
116,Andrés Iniesta Luján,Shot,21.0,1032.0,0.020349,5216,24,"[65.958273381295, 24.63075539568345]",7,1111111111,35,"[120, 40]",11
117,Sergi Roberto Carnicer,Shot,12.0,1128.0,0.010638,6379,23,"[61.67888888888892, 61.55380952380949]",8,1111111111,35,"[120, 40]",11
118,Samuel Yves Umtiti,Shot,12.0,1187.0,0.010110,5492,23,"[41.16050724637682, 25.78876811594206]",9,1111111111,35,"[120, 40]",11


In [ ]:
df_starting_pass.to_csv('df_starting_pass.csv', index=False, encoding='utf-8-sig')

In [ ]:
# creating state transition matrix

pivot_table = df_starting_pass.pivot(index='row_id_x', columns='row_id_y', values='Pass Probability')
state_tr_matrix = pivot_table.to_numpy()
state_tr_matrix = np.nan_to_num(state_tr_matrix, nan=0)

# extra row for row id 11
new_row = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
new_arr = np.zeros((state_tr_matrix.shape[0] + 1, state_tr_matrix.shape[1]))

new_arr[:-1,:] = state_tr_matrix
new_arr[-1,:] = new_row

P = new_arr

In [ ]:
# function for setting priorities in case of player passing to himself, or same passing sequence repeats over and over

def requirements(lst, val):
    if val in lst:
        return False
    else:
        return True
    

In [ ]:
# Markov application

# finding steady state matrix by power iteration
Pm = P ** 50

# normalizing the rows

np.fill_diagonal(Pm, 0.0)
last_row = Pm.shape[0] - 1
last_col = Pm.shape[1] - 1

Pm[last_row, :] = 0.0
Pm[last_row, last_col] = 1.0

row_sums = Pm.sum(axis=1)
Pm = Pm / row_sums[:, np.newaxis]

# initiating the sequence from goalkeeper

n = Pm.shape[0] # number of nodes
pi = np.zeros(n)
pi[0] = 1 # set the probability of first node (goalkeeper) to 1

current_node = 0 # we need to add some sort of identifier for the goalkeeper in the dataframe "df_starting" in order to initiate the sequence
route = [current_node]


cnt = 0

while current_node != 11 and cnt < 1580: # continue until reaching node 12
    cnt += 1
    
    pi = np.dot(pi, Pm)
    pi_sorted = np.sort(pi)[::-1]

    for i in range(len(pi_sorted)):
        
        ind = np.where(pi == pi_sorted[i])[0][0]
        next_node = ind

        if len(route) > 1:
        
            if requirements(route, ind):
                next_node = ind
                break
            
        else:
            break      

    route.append(next_node)
    current_node = next_node
    
    n = Pm.shape[0]
    pi = np.zeros(n)
    pi[current_node] = 1
    
    
print(route)


[0, 4, 8, 3, 1, 11]


In [ ]:
ratio_dict = {}

col_totals = np.sum(Pm, axis=0)
col_totals

for i in range(len(col_totals)):
    ratio_dict[i] = col_totals[i]

ratio_dict


{0: 0.001383607886016711,
 1: 3.0400433573925887,
 2: 8.936685483754802e-11,
 3: 1.1202410096529656,
 4: 0.999999480109635,
 5: 1.99999999547946,
 6: 1.4967359908971245e-05,
 7: 0.9999999986623499,
 8: 0.8827786527939315,
 9: 0.01545705075307293,
 10: 5.0157538057932954e-17,
 11: 2.9400818798207036}

In [ ]:
np.sum(Pm, axis=0)


array([1.38360789e-03, 3.04004336e+00, 8.93668548e-11, 1.12024101e+00,
       9.99999480e-01, 2.00000000e+00, 1.49673599e-05, 9.99999999e-01,
       8.82778653e-01, 1.54570508e-02, 5.01575381e-17, 2.94008188e+00])

In [ ]:
nodes_route = set(route)

occurances = {}
for elem in route:
    occurances[elem] = ratio_dict[elem]

route_adj = [route[0], route[1]]

occurances_copy = occurances.copy()
  
del occurances_copy[route[0]]
del occurances_copy[route[1]]
del occurances_copy[route[-1]]
del occurances_copy[route[-2]]

if len(nodes_route) > 7:
  
    sorted_dict = sorted(occurances_copy.items(), key=lambda x: x[1], reverse=True)
    # Get the keys of the 3 highest values
    top_keys = [k for k, v in sorted_dict[:3]]
    # Loop through the route list and insert the top keys by their first appearance
  
    while len(route_adj) < 5:
  
        for i in range(2, len(route)-2):
            if route[i] in top_keys:
                route_adj.append(route[i])
                top_keys.remove(route[i])
            

    route_adj.append(route[-2])
    route_adj.append(route[-1])
    
else:
    for i in range(2, len(route)-2):
        if route[i] in occurances_copy: 
            route_adj.append(route[i])

    route_adj.append(route[-2])
    route_adj.append(route[-1])

    


print("Most probable route:", route_adj)

Most probable route: [0, 4, 8, 3, 1, 11]


In [ ]:
df_starting

,player_id,season_starts,average_location,player_from,row_id
0,20055,35,"[11.75150684931507, 40.30493150684933]",Marc-André ter Stegen,0
1,5503,32,"[75.0770290964777, 45.517151607963264]",Lionel Andrés Messi Cuccittini,1
2,5246,30,"[81.11993670886076, 32.33829113924053]",Luis Alberto Suárez Díaz,2
3,5470,29,"[58.1833333333333, 46.89243421052626]",Ivan Rakitić,3
4,5213,28,"[40.62254601226996, 53.52331288343562]",Gerard Piqué Bernabéu,4
5,5211,28,"[58.5618181818182, 13.467972027972031]",Jordi Alba Ramos,5
6,5203,28,"[56.009207161125325, 37.920204603580565]",Sergio Busquets i Burgos,6
7,5216,24,"[65.958273381295, 24.63075539568345]",Andrés Iniesta Luján,7
8,6379,23,"[61.67888888888892, 61.55380952380949]",Sergi Roberto Carnicer,8
9,5492,23,"[41.16050724637682, 25.78876811594206]",Samuel Yves Umtiti,9


In [ ]:
# creating route dataframe

selected_rows = []

for i in route_adj:
    selected_row = df_starting.loc[df_starting['row_id'] == i]
    selected_rows.append(selected_row)

route_df = pd.concat(selected_rows)
route_df['node_radius_ratio'] = route_df['row_id'].map(ratio_dict)

min_value = route_df['node_radius_ratio'].min()
max_value = route_df['node_radius_ratio'].max()

# normalize the column values between 1 and 2
route_df['node_radius_ratio'] = 1 + (route_df['node_radius_ratio'] - min_value) / (max_value - min_value)
route_df.loc[route_df['row_id'] == 11, 'node_radius_ratio'] = 1

#print(route_df)

shifted_df = route_df.shift(-1)

# concatenate the original DataFrame and the shifted DataFrame horizontally
concatenated_df = pd.concat([route_df, shifted_df], axis=1)

# rename the new columns
concatenated_df.columns = ['player_id', 'season_starts', 'average_location', 'player_from', 'row_id', 'node_radius_ratio',
                           'player_id_2', 'season_starts_2', 'average_location_2', 'player_from_2', 'row_id_2', 'node_radius_ratio_2']

# fill the NaN values in the last row with the first row values
concatenated_df.iloc[-1, -6:] = concatenated_df.iloc[-1, :6].values

concatenated_df

,player_id,season_starts,average_location,player_from,row_id,node_radius_ratio,player_id_2,season_starts_2,average_location_2,player_from_2,row_id_2,node_radius_ratio_2
0,20055,35,"[11.75150684931507, 40.30493150684933]",Marc-André ter Stegen,0,1.000000,5.213000e+03,28.0,"[40.62254601226996, 53.52331288343562]",Gerard Piqué Bernabéu,4.0,1.328637
4,5213,28,"[40.62254601226996, 53.52331288343562]",Gerard Piqué Bernabéu,4,1.328637,6.379000e+03,23.0,"[61.67888888888892, 61.55380952380949]",Sergi Roberto Carnicer,8.0,1.290060
8,6379,23,"[61.67888888888892, 61.55380952380949]",Sergi Roberto Carnicer,8,1.290060,5.470000e+03,29.0,"[58.1833333333333, 46.89243421052626]",Ivan Rakitić,3.0,1.368208
3,5470,29,"[58.1833333333333, 46.89243421052626]",Ivan Rakitić,3,1.368208,5.503000e+03,32.0,"[75.0770290964777, 45.517151607963264]",Lionel Andrés Messi Cuccittini,1.0,2.000000
1,5503,32,"[75.0770290964777, 45.517151607963264]",Lionel Andrés Messi Cuccittini,1,2.000000,1.111111e+09,35.0,"[120, 40]",Shot,11.0,1.000000
11,1111111111,35,"[120, 40]",Shot,11,1.000000,1.111111e+09,35.0,"[120, 40]",Shot,11.0,1.000000


In [ ]:
# creating csv files

def writenodes(df):
    ndf = df.copy()
    ndf['x_pos_abs'] = ndf['average_location'].apply(lambda x: x[0])
    ndf['y_pos_abs'] = ndf['average_location'].apply(lambda x: x[1])
    ndf['x_pos_perc'] = ndf['x_pos_abs'] / 120
    ndf['y_pos_perc'] = ndf['y_pos_abs'] / 80
    ndf['name'] = ndf['player_from']

    return ndf.to_csv('player_nodes.csv', index=False, encoding='utf-8-sig')


def writelinks(df):
    ndf = df.copy()
    
    freq_max = ndf['Frequency'].max()
    freq_min = ndf['Frequency'].min()

    ndf['pass_completed'] = ((ndf['Frequency'] - freq_min) / (freq_max - freq_min)) * 4
    ndf['source'] = ndf['Passing']
    ndf['target'] = ndf['Receiving']
    ndf['source_x'] = ndf['average_location_x'].apply(lambda x: x[0]/120)
    ndf['source_y'] = ndf['average_location_x'].apply(lambda x: x[1]/80)
    ndf['target_x'] = ndf['average_location_y'].apply(lambda x: x[0]/120)
    ndf['target_y'] = ndf['average_location_y'].apply(lambda x: x[1]/80)


    return ndf.to_csv('player_links.csv', index=False, encoding='utf-8-sig')

def writeroute(df):
    ndf = df.copy()
    ndf['source'] = ndf['player_from']
    ndf['name'] = ndf['player_from']
    ndf['target'] = ndf['player_from_2']
    ndf['source_x'] = ndf['average_location'].apply(lambda x: x[0]/120)
    ndf['source_y'] = ndf['average_location'].apply(lambda x: x[1]/80)
    ndf['target_x'] = ndf['average_location_2'].apply(lambda x: x[0]/120)
    ndf['target_y'] = ndf['average_location_2'].apply(lambda x: x[1]/80)

    return ndf.to_csv('player_route.csv', index=False, encoding='utf-8-sig')


writenodes(df_starting)
writelinks(df_starting_pass)
writeroute(concatenated_df)

# Random Code/Test Area

In [ ]:
events.keys()

dict_keys(['starting_xis', 'half_starts', 'passes', 'ball_receipts', 'carrys', 'pressures', 'interceptions', 'dispossesseds', 'duels', 'miscontrols', 'ball_recoverys', 'dribbled_pasts', 'dribbles', 'clearances', 'shots', 'goal_keepers', 'blocks', 'foul_committeds', 'foul_wons', 'shields', 'half_ends', 'substitutions', 'injury_stoppages', 'referee_ball_drops', 'tactical_shifts', 'own_goal_againsts', 'own_goal_fors', 'bad_behaviours', '50/50s', 'offsides', 'errors', 'player_offs', 'player_ons', 'Starting XI'])

In [ ]:
# df with all shot events
shots_df = events["shots"]
shots_df.info()
shots_df['shot_outcome'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839 entries, 0 to 838
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     839 non-null    object 
 1   index                  839 non-null    int64  
 2   period                 839 non-null    int64  
 3   timestamp              839 non-null    object 
 4   minute                 839 non-null    int64  
 5   second                 839 non-null    int64  
 6   type                   839 non-null    object 
 7   possession             839 non-null    int64  
 8   possession_team        839 non-null    object 
 9   play_pattern           839 non-null    object 
 10  team                   839 non-null    object 
 11  player                 839 non-null    object 
 12  position               839 non-null    object 
 13  location               839 non-null    object 
 14  duration               839 non-null    float64
 15  relate

array(['Off T', 'Blocked', 'Saved', 'Goal', 'Wayward', 'Post',
       'Saved Off Target', 'Saved to Post'], dtype=object)

In [ ]:
# Finding average location of all players at shot event

# Copy avgloc function but remove play pattern constraint
def shotavgloc(id,event_df):
  events = event_df.loc[(event_df['player_id'] == id)].reset_index(drop=True)
  locations = events['location']
  return [sum(x)/len(x) for x in zip(*locations)]

# Copy player/location df but with locations reflecting shooting events
avgshotlocs = [shotavgloc(id, shots_df) for id in top_starts]
shooters = pd.DataFrame(list(zip(top_starts,num_starts, avgshotlocs)), columns = ['player_id', 'season_starts', 'average_location'])
shooters['player_from'] = df['player_id'].map(player_dict)
shooters

,player_id,season_starts,average_location,player_from
0,5211,33,"[102.52499999999999, 31.162500000000005]",Jordi Alba Ramos
1,5503,33,"[102.77948717948712, 39.63897435897437]",Lionel Andrés Messi Cuccittini
2,8118,32,"[109.72000000000001, 42.81333333333334]",Frenkie de Jong
3,5203,30,"[106.55, 40.2]",Sergio Busquets i Burgos
4,20055,29,"[102.0, 41.1]",Marc-André ter Stegen
5,5487,29,"[107.52031249999997, 37.734375000000014]",Antoine Griezmann
6,6826,28,"[107.63, 37.03]",Clément Lenglet
7,30486,26,"[105.62727272727271, 37.849999999999994]",Pedro González López
8,21881,21,"[106.75714285714288, 48.82142857142856]",Sergino Dest
9,43728,20,"[108.23333333333333, 40.98888888888889]",Óscar Mingueza García


In [ ]:
# Example with all of Messi's passes
messi_passes = passes_df.loc[passes_df['player_id'] == 5503].reset_index(drop=True)
# messi_passes

In [ ]:
# Example with all of Messi's shots
messi_shots = shots_df.loc[shots_df['player_id'] == 5503].reset_index(drop=True)
messi_shots

In [ ]:
# Dictionary and dataframe of passer and recipient pairs
passes = defaultdict(int)
passers = complete_passes_df['player']
recipients = complete_passes_df['pass_recipient']
merged_list = list(zip(passers, recipients))

for element in merged_list:
  passes[element] += 1

dfPass = pd.DataFrame([[k[0],k[1],v] for k,v in passes.items()]) \
           .rename(columns={0:'Passing',1:'Receiving',2:'Frequency'})


print(passes)
print(dfPass.shape)
dfPass.head(5)

defaultdict(<class 'int'>, {('Lionel Andrés Messi Cuccittini', 'Sergio Busquets i Burgos'): 256, ('Sergio Busquets i Burgos', 'Moriba Kourouma Kourouma'): 47, ('Moriba Kourouma Kourouma', 'Ronald Federico Araújo da Silva'): 26, ('Ronald Federico Araújo da Silva', 'Marc-André ter Stegen'): 85, ('Marc-André ter Stegen', 'Clément Lenglet'): 168, ('Clément Lenglet', 'Pedro González López'): 126, ('Pedro González López', 'Marc-André ter Stegen'): 23, ('Marc-André ter Stegen', 'Ronald Federico Araújo da Silva'): 61, ('Ronald Federico Araújo da Silva', 'Sergio Busquets i Burgos'): 59, ('Sergio Busquets i Burgos', 'Ousmane Dembélé'): 55, ('Ronald Federico Araújo da Silva', 'Clément Lenglet'): 124, ('Clément Lenglet', 'Jordi Alba Ramos'): 535, ('Jordi Alba Ramos', 'Gerard Piqué Bernabéu'): 51, ('Gerard Piqué Bernabéu', 'Moriba Kourouma Kourouma'): 13, ('Ronald Federico Araújo da Silva', 'Gerard Piqué Bernabéu'): 66, ('Gerard Piqué Bernabéu', 'Ronald Federico Araújo da Silva'): 54, ('Marc-André 

,Passing,Receiving,Frequency
0,Lionel Andrés Messi Cuccittini,Sergio Busquets i Burgos,256
1,Sergio Busquets i Burgos,Moriba Kourouma Kourouma,47
2,Moriba Kourouma Kourouma,Ronald Federico Araújo da Silva,26
3,Ronald Federico Araújo da Silva,Marc-André ter Stegen,85
4,Marc-André ter Stegen,Clément Lenglet,168


In [ ]:
# Shots and goals of each player
players = []
for player in shooters['player_from']:
  df = shots_df.loc[shots_df['player'] == player]
  shots = df.shape[0]
  goals = df.loc[df['shot_outcome'] == 'Goal'].shape[0]
  players.append((player,shots,goals))
shots = pd.DataFrame(list(players),
               columns =['Player', 'Shots',"Goals"])
shots

,Player,Shots,Goals
0,Jordi Alba Ramos,24,3
1,Lionel Andrés Messi Cuccittini,195,30
2,Frenkie de Jong,15,2
3,Sergio Busquets i Burgos,2,0
4,Marc-André ter Stegen,1,0
5,Antoine Griezmann,64,12
6,Clément Lenglet,10,1
7,Pedro González López,22,3
8,Sergino Dest,14,2
9,Óscar Mingueza García,9,2


In [ ]:
#shots_df.loc[shots_df['player'] == 'Marc-André ter Stegen']
shots_df.play_pattern.unique()

array(['From Corner', 'Regular Play', 'From Counter', 'From Free Kick',
       'From Throw In', 'From Goal Kick', 'From Keeper', 'Other',
       'From Kick Off'], dtype=object)